<h3>
    <b>
        <font color='#660000'>
            Métodos Iterativos para Sistemas Lineares
        </font>
    </b>
</h3>
<b>Livro:</b><i> Álgebra Linear e suas Aplicações: Notas de Aula</i>, Petronio Pulino. 
<h4>
    <b>
        1. Iteração de Ponto Fixo. 
    </b>
</h4>
    

<br>
    Seja $A \in \mathbb{R}^{n \times n}$ uma matriz invertível e $b \in \mathbb{R}^n$. Considere o Sistema Linear: encontrar $x^{*} \in \mathbb{R}^n$ solução da equação $Ax=b$. Como $A$ é invertível, o sistema linear $Ax=b$ possui única solução, que vamos denotar por $x^* = A^{-1}b$. Podemos escrever o sistema linear em uma forma equivalente 
    $$x=Px+d. $$
Desse modo, um método iterativo consiste em considerar uma aproximação inicial $x^{(0)} \in \mathbb{R}^n$, para a solução $x^*$ e construir uma sequência 
$$x^{(k+1)}=Px^{(k)}+d \quad \text{para} \quad k=0,1,2,\dots. $$

<h4>
    <b>
        2. Matriz Estritamente Diagonalmente Dominante por Linhas
    </b>
</h4>
<b>Definição:</b> Dizemos que $A = [a_{ij}] \in \mathbb{R}^{n\times n}$ é uma matriz <b>Estritamente Diagonalmente Dominante por Linhas</b> se

$$ |a_{ij}| > \sum_{j=1,\ j \neq i} |a_{ij}|\ ;\ i=1,\dots,n.$$

A função a seguir verifica se uma matriz é Estritamente Diagonalmente Dominante por Linhas, retornando <b>True</b> se a é, e retornando <b>False</b> caso contrário.

In [1]:
import numpy as np

def diag_dominante(A):

    n = A.shape[0]
    
    for i in range(n):
    
        row_soma = np.soma(np.abs(A[i, :])) - np.abs(A[i, i])
        
        if np.abs(A[i, i]) <= row_soma:
            return False
            
    return True

<h4>
    <b>
        3. Fatoração de Cholesky (Via NumPy)
    </b>
</h4>

A função a seguir retorna True se existe fatoração de Cholesky de uma matriz $A$ e retorna False caso contrário.

In [24]:
import numpy as np

def verifica_cholesky(A):
    try:                         # Tenta calcular a fatoração de Cholesky
        np.linalg.cholesky(A)
        return True              # Sucesso: a matriz admite Cholesky
    except np.linalg.LinAlgError:
        return False             # Erro: a matriz não admite Cholesky

<h4>
    <b>
        4. Método de Jacobi
    </b>
</h4>

In [21]:
import numpy as np

def jacobi(A, b, x0, tol, N=1000):
    
    x = x0
    
    n = len(b)
    
    for k in range(N):
        
        x_novo = x.copy()                       # Cria uma nova aproximação baseada no estado atual.
        
        for i in range(n):                      # Para cada linha i.
            soma = 0
            for j in range(i):                  # Soma antes da diagonal.
                soma += A[i, j] * x[j]
            for j in range(i + 1, n):           # Soma depois da diagonal.
                soma += A[i, j] * x[j]

            x_novo[i] = (b[i] - soma) / A[i, i] # Atualiza x_i^{(k+1)}
        
        
        #if np.linalg.norm(x_novo - x, ord=np.inf) < tol: # Critério de parada: Cauchy   (noma sup).
        if np.linalg.norm(b - A @ x, ord=np.inf) < tol:   # Critério de parada: Residual (norma sup)
            print('Convergiu em {} iterações.'.format(k+1)) 
            retuse n só vou p aula do morn x_novo, k + 1
        
        x = x_novo  # Atualiza a solução para a próxima iteração.
    
    raise ValueError("O método de Jacobi não convergiu dentro do número máximo de iterações.")            

<h4>
    <b>
        <font color='#176303'>Exercício 4.1.</font>
    </b>
</h4>
Encontre uma aproximação pelo Método de Jacobi para o sistema linear

$$ Ax=b \Rightarrow \begin{bmatrix}
 5 & 2 & 1  & 1 \\
 2 & 6 & 2 & 1 \\
 1& 2 & 7 & 2 \\
1 &1  &2  & 8 \\
\end{bmatrix}\begin{bmatrix}
 x_1\\
 x_2\\
 x_3\\
x_4
\end{bmatrix}=\begin{bmatrix}
 29\\
 31\\
 26\\
19
\end{bmatrix} $$

Vajamos se $A$ é Estritamente Diagonalmente Dominante:

In [3]:
import numpy as np

A = np.array([[5,2,1,1],[2,6,2,1],[1,2,7,2],[1,1,2,8]]) # Declarando a matriz A.

diag_dominante(A) # Verificando se A é estritamente dominante.

True

Pelo resultado anterior (True) o Método de Jacobi converge para qualquer aproximação inicial $x^{(0)}$.

In [26]:
b  = np.array([29,31,26,19])
x0 = np.array([1,1,1,1])
x  = jacobi(A, b, x0, 10e-5)
x  = x[0]
x

Convergiu em 4 iterações.


array([4, 3, 2, 1])

Vejamos o resíduo:

In [23]:
R = np.linalg.norm(b - A @ x, ord=np.inf)
R

0.0

A solução é exata.

<h4>
    <b>
        5. Método de Gauss-Seidel
    </b>
</h4>

In [32]:
import numpy as np

def gauss_seidel(A, b, x0, tol, max_iter=100):
   
    n = len(b)
    x = np.copy(x0)  
    
    for k in range(max_iter):
        x_anterior = np.copy(x)
        
        for i in range(n):
            
            soma1 = sum(A[i, j] * x[j] for j in range(i)) # Soma com valores mais atualizados (j < i)
            
            soma2 = sum(A[i, j] * x_anterior[j] for j in range(i + 1, n)) # Soma com valores da iteração anterior (j > i)
            
            # Atualiza x[i] usando a fórmula de Gauss-Seidel
            x[i] = (b[i] - soma1 - soma2) / A[i, i]
        
        # Verifica o critério de parada (norma infinita)
        if np.linalg.norm(x - x_anterior, ord=np.inf) < tol:
            print('Convergiu em {} iterações.'.format(k+1)) 
            return x, k + 1  # Retorna solução e número de iterações
    
    raise ValueError("O método não convergiu após o número máximo de iterações")

<h4>
    <b>
        <font color='#176303'>Exercício 5.1.</font>
    </b>
</h4>
Resolva o problema anterior pelo Método de Gauss-Seidel usando $\varepsilon = 10^{-5}$. Quantas iterações foram necessárias?

Sol: Vamos verificar se $A$ é simétrica e positiva definida.

In [28]:
import numpy as np

A = np.array([[5,2,1,1],[2,6,2,1],[1,2,7,2],[1,1,2,8]]) # Declarando a matriz A.

verifica_cholesky(A)

True

A matriz é simétrica e positiva definida. Logo, o Método de Gauss-Seidel converge para a solução independente da escolha do vetor inicial $x^{(0)}$.

In [33]:
b  = np.array([29,31,26,19])
x0 = np.array([1,1,1,1])
x  = gauss_seidel(A, b, x0, 10e-5)
x  = x[0]
x

Convergiu em 3 iterações.


array([4, 3, 2, 1])

<h4>
    <b>
        6. Método de Sobrerrelaxação 
    </b>
</h4>

In [63]:
import numpy as np
import pandas as pd

def sobrerrelaxacao(A, b, omega, num_iter=100, tol=10e-5):

    n = A.shape[0]  # Tamanho da matriz
    x = np.zeros(n)  # Vetor de soluções inicializado em zeros

    # Iterações
    for k in range(num_iter):
        x_anterior = np.copy(x)  # Copia do vetor solução antes da iteração

        for i in range(n):
            # Primeiramente, calcula x_i(k+1) sem a contribuição de x_i
            soma1 = 0
            for j in range(i):
                soma1 += A[i, j] * x[j]  # Soma dos produtos com os x_j(k+1)

            soma2 = 0
            for j in range(i + 1, n):
                soma2 += A[i, j] * x[j]  # Soma dos produtos com os x_j(k)

            # Atualiza x_i(k+1) usando a fórmula de Sobrerrelaxação
            x[i] = omega * (b[i] - soma1 - soma2) / A[i, i] + (1 - omega) * x[i]

        # Verificação de convergência: norma infinita do erro
        erro = np.linalg.norm(x - x_anterior, np.inf)
        if erro < tol:
            print(f"Convergência atingida na iteração {k + 1} com erro {erro:.6e}")
            return x, k + 1  # Retorna a solução e o número de iterações
    
    raise RuntimeError(f"O método não convergiu após {num_iter} iterações.")

<h4>
    <b>
        <font color='#176303'>Exercício 6.1.</font>
    </b>
</h4>
Utilize alguns valores no Exercício  4.1  para $\omega$ e mostre qual valor de $\omega$ é tal que o método SOR realiza menos iterações.

In [57]:
A     = np.array([[5,2,1,1],[2,6,2,1],[1,2,7,2],[1,1,2,8]]) # Declarando a matriz A.
b     = np.array([29,31,26,19])
omega = 1
x     = sobrerrelaxacao(A, b, omega)
x     = x[0]
x

Convergência atingida na iteração 8 com erro 3.255367e-05


array([4.00000101, 2.99999919, 2.00000149, 0.9999996 ])

In [72]:
A        = np.array([[5,2,1,1],[2,6,2,1],[1,2,7,2],[1,1,2,8]]) 
b        = np.array([29,31,26,19])
omegas   = [0.1, 0.5, 0.8, 1, 1.1, 1.2, 1.5, 1.7, 2]

solucoes  = np.array([])
iteracoes = np.array([])
residuo   = np.array([])

for omega in omegas:

    try:
        x = sobrerrelaxacao(A, b, omega)
        solucoes  = np.append(solucoes,  x[0])
        iteracoes = np.append(iteracoes, x[1])    
        residuo   = np.append(residuo,   np.linalg.norm(b - A @ x[0], ord=np.inf))
        
    except RuntimeError as e:
        solucoes  = np.append(solucoes,  '-')
        iteracoes = np.append(iteracoes, '-')
        residuo   = np.append(residuo,   '-')

df = pd.DataFrame({
    'Omega': omegas,
    'Iterações': iteracoes,
    'Resíduo': residuo
})

df

Convergência atingida na iteração 86 com erro 9.580961e-05
Convergência atingida na iteração 14 com erro 8.284933e-05
Convergência atingida na iteração 10 com erro 5.684951e-05
Convergência atingida na iteração 8 com erro 3.255367e-05
Convergência atingida na iteração 8 com erro 8.106719e-05
Convergência atingida na iteração 11 com erro 2.346646e-05
Convergência atingida na iteração 21 com erro 4.905124e-05
Convergência atingida na iteração 38 com erro 6.933642e-05


,Omega,Iterações,Resíduo
0,0.1,86.0,0.004573733037137373
1,0.5,14.0,0.0005029293592002659
2,0.8,10.0,0.0001668107248526951
3,1.0,8.0,9.024179341565741e-06
4,1.1,8.0,0.00018121304818663475
5,1.2,11.0,4.386383447751996e-05
6,1.5,21.0,0.0001383912292602929
7,1.7,38.0,0.00020450955977224794
8,2.0,-,-
